Upload data to sqlite tables.

In [18]:
import pandas as pd
import numpy as np
from IPython import display as dis
import scipy.io.wavfile as wav
import tensorflow as tf
import time

%matplotlib inline

In [2]:
dis.Audio("dataset/wav/Ses01F_impro01/Ses01F_impro01_M007.wav")

In [3]:
(rate,sig) = wav.read("dataset/wav/Ses01F_impro01/Ses01F_impro01_F000.wav")
print(sig)
print(sig.shape)

[-165 -163 -125 ...,  -87 -104 -137]
(31129,)


In [42]:
class network(object):
    
    input_dim = 31129
    classes = 2
    hidden_encoder_dim = 1000
    hidden_layers = 1
    latent_dim = 61

    def build_layers(self):
        tf.reset_default_graph()
        
        input_dim = self.input_dim

        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, input_dim])
            self.keep_prob = tf.placeholder("float")
            self.lr = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            
            #hidden_encoder = tf.layers.dense(self.x, latent_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            #hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            
        with tf.variable_scope("Layer_Dense_Softmax"):
            self.y = tf.layers.dense(hidden_encoder, input_dim, activation=None)
            
        with tf.variable_scope("Loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = self.y))
            #loss = tf.losses.mean_squared_error(labels = self.y_, predictions = self.y)
            #loss = tf.clip_by_value(loss, -1e-1, 1e-1)
            #loss = tf.where(tf.is_nan(loss), 1e-1, loss)
            #loss = tf.where(tf.equal(loss, -1e-1), tf.random_normal(loss.shape), loss)
            #loss = tf.where(tf.equal(loss, 1e-1), tf.random_normal(loss.shape), loss)
            
            self.regularized_loss = loss
            correct_prediction = tf.equal(tf.argmax(self.y_, 1), tf.argmax(self.y, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")
            
        with tf.variable_scope("Optimizer"):
            learning_rate=self.lr
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [43]:
import collections

class Train:    
    
    def train(epochs, net, lrs):
   
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            start_time = time.perf_counter()
            
            
            for c, lr in enumerate(lrs):
                for epoch in range(1, (epochs+1)):
                    print("Step {} ".format(epoch))
                    _, train_loss = sess.run([net.train_op, net.regularized_loss], #net.summary_op
                                              feed_dict={net.x: x_train, 
                                                         net.y_: y_train,
                                                         net.keep_prob: 1,
                                                         net.lr:lr})

                    print("Training Loss: {:.6f}".format(train_loss))

                    #valid_accuracy,valid_loss = sess.run([net.tf_accuracy, net.regularized_loss], #net.summary_op 
                    #                                      feed_dict={net.x_input: x_valid[np.newaxis,...], 
                    #                                                 net.y_input_: y_valid[np.newaxis,...],
                    #                                                 net.lr:lr})
                        
                    #accuracy, y_pred = sess.run([net.tf_accuracy, 
                    #                           net.pred, 
                    #                           net.actual, net.y], 
                    #                          feed_dict={net.x_input: x_test[np.newaxis,...], 
                    #                                     net.y_input_: y_test[np.newaxis,...],
                    #                                     net.lr:lr})


In [58]:
import itertools

class Hyperparameters:
    def start_training():
        
        epochs = 10
        lrs = [1e-5]
        n = network()
        n.build_layers()
        Train.train(epochs, n, lrs)
            

In [59]:
x_train = y_train = np.reshape(sig, (1, -1))
Hyperparameters.start_training()

Step 1 
Training Loss: -1909006.375000
Step 2 
Training Loss: -30779968.000000
Step 3 
Training Loss: -59287088.000000
Step 4 
Training Loss: -88266592.000000
Step 5 
Training Loss: -117741392.000000
Step 6 
Training Loss: -147897248.000000
Step 7 
Training Loss: -178906256.000000
Step 8 
Training Loss: -210892048.000000
Step 9 
Training Loss: -244253120.000000
Step 10 
Training Loss: -279045792.000000
